In [1]:
from Policy1_demo_workloads import wrk_different_requests
from iDynamicsPackagesModules.SchedulingPolicyExtender.my_cluster_utils import (
    gather_all_nodes, build_nodeinfo_objects,
    build_podinfo_objects, gather_all_pods)

from iDynamicsPackagesModules.GraphDynamicsAnalyzer.graph_builder import build_call_graph
from iDynamicsPackagesModules.Evaluations.Policy1_eval_Graph_dynamics.Policy1_Call_Graph_Aware import Policy1CallGraphAware
from iDynamicsPackagesModules.Evaluations.Policy2_eval_Delay_dynamics.Policy2_Latency_Aware import Policy2LatencyAware
from iDynamicsPackagesModules.Evaluations.Policy3_eval_Bandwidth_dynamics.Policy3_Bandwidth_Aware import Policy3BandwidthAware




Pods already exist in the namespace. Skipping DaemonSet deployment.
{'k8s-worker-6': {'k8s-worker-6': None, 'k8s-worker-3': 8.501999999999999, 'k8s-worker-5': 10.609, 'k8s-worker-1': 7.439, 'k8s-worker-9': 15.584000000000001, 'k8s-worker-2': 23.539, 'k8s-worker-7': 13.525, 'k8s-worker-8': 10.648, 'k8s-worker-4': 18.713}, 'k8s-worker-9': {'k8s-worker-9': None, 'k8s-worker-3': 3.558, 'k8s-worker-7': 11.57, 'k8s-worker-5': 19.683, 'k8s-worker-6': 15.571, 'k8s-worker-1': 3.5599999999999996, 'k8s-worker-4': 15.710999999999999, 'k8s-worker-8': 7.644, 'k8s-worker-2': 11.396999999999998}, 'k8s-worker-3': {'k8s-worker-3': None, 'k8s-worker-7': 3.714, 'k8s-worker-9': 3.48, 'k8s-worker-5': 4.622, 'k8s-worker-4': 2.785, 'k8s-worker-6': 8.597000000000001, 'k8s-worker-8': 16.817, 'k8s-worker-1': 0.565, 'k8s-worker-2': 9.568999999999999}, 'k8s-worker-7': {'k8s-worker-7': None, 'k8s-worker-3': 3.625, 'k8s-worker-6': 13.473, 'k8s-worker-8': 18.785, 'k8s-worker-5': 11.484, 'k8s-worker-1': 6.481, 'k8s-wo

In [2]:


request_A_script_path = "/home/ubuntu/DeathStarBench/socialNetwork/wrk2/scripts/social-network/compose-post.lua"
request_B_script_path = "/home/ubuntu/DeathStarBench/socialNetwork/wrk2/scripts/social-network/read-home-timeline.lua"
request_C_script_path = "/home/ubuntu/DeathStarBench/socialNetwork/wrk2/scripts/social-network/read-user-timeline.lua"

request_mix_script_path = "/home/ubuntu/DeathStarBench/socialNetwork/wrk2/scripts/social-network/mixed-workload.lua" # default mix percentage: 60% A, 30% B, 10% C

script_path = [
    request_A_script_path,
    request_B_script_path,
    request_C_script_path,
    request_mix_script_path
]

url = [
    #diffferent namespace of social-network Application
    "http://nginx-thrift.social-network.svc.cluster.local:8080"       # in namespace of "social-network"
#     "http://nginx-thrift.social-network2.svc.cluster.local:8080",
#    "http://nginx-thrift.social-network3.svc.cluster.local:8080"
]

each_wrk_duration = "30s" # eg., "30s", "1m", 5m", "10m", "15m"


for i in range(0, len(script_path)):
    wrk_different_requests(script_path[i], url[0], request_interval = each_wrk_duration)
        # 1) Gather cluster data
    raw_nodes = gather_all_nodes()
    candidate_nodes = build_nodeinfo_objects(raw_nodes)

    # 1) Gather cluster data
    raw_nodes = gather_all_nodes()
    candidate_nodes = build_nodeinfo_objects(raw_nodes)

    # Suppose your social-network microservices are in namespace "social-network"
    raw_pods = gather_all_pods(namespace="social-network")
    pods = build_podinfo_objects(raw_pods)

    # 2) (Optional) Build a call graph from 'graph_builder.py'
    #    This might be used by Policy1 to place heavily communicating microservices.
    G = build_call_graph(namespace="social-network")
    # Convert the NetworkX graph to a simple dict: (src, dst) -> traffic_kb
    traffic_matrix = {}
    for (u, v, data) in G.edges(data=True):
        traffic_matrix[(u, v)] = data.get("weight", 0.0)  # store as KB

    # 3) Initialize each policy with scenario-specific config
    policy1 = Policy1CallGraphAware()
    policy1.initialize_policy(config={"traffic_matrix": traffic_matrix})

    policy2 = Policy2LatencyAware()
    policy2.initialize_policy(config={"latency_threshold": 5.0})  # example

    policy3 = Policy3BandwidthAware()
    policy3.initialize_policy(config={"high_traffic_threshold": 300.0})  # example

    # 4) Choose which scenario you want to test
    # Scenario 1: call-graph dynamics => we expect policy1 to shine
    print("=== Scenario1: Call-Graph_Aware ===")
    decisions_s1 = policy1.schedule_all(pods, candidate_nodes)
    for dec in decisions_s1:
        print(f"  Pod {dec.pod_name} -> Node {dec.selected_node}")

    # Scenario 2: cross-node delays => policy2 might do better
    print("\n=== Scenario2: Latency_Aware ===")
    decisions_s2 = policy2.schedule_all(pods, candidate_nodes)
    for dec in decisions_s2:
        print(f"  Pod {dec.pod_name} -> Node {dec.selected_node}")

    # Scenario 3: cross-node bandwidth changes => policy3 might do better
    print("\n=== Scenario3: Bandwidth_Aware ===")
    decisions_s3 = policy3.schedule_all(pods, candidate_nodes)
    for dec in decisions_s3:
        print(f"  Pod {dec.pod_name} -> Node {dec.selected_node}")



Running wrk_different_requests test for url: http://nginx-thrift.social-network.svc.cluster.local:8080
Running command: /home/ubuntu/DeathStarBench/wrk2/wrk -D exp -t4 -c100 -d30s -L -s /home/ubuntu/DeathStarBench/socialNetwork/wrk2/scripts/social-network/compose-post.lua http://nginx-thrift.social-network.svc.cluster.local:8080 -R20
Running 30s test @ http://nginx-thrift.social-network.svc.cluster.local:8080
  4 threads and 100 connections

  Thread calibration: mean lat.: 272.632ms, rate sampling interval: 695ms
  Thread calibration: mean lat.: 273.149ms, rate sampling interval: 619ms
  Thread calibration: mean lat.: 287.898ms, rate sampling interval: 845ms
  Thread calibration: mean lat.: 278.456ms, rate sampling interval: 720ms

-----------------------------------------------------------------------
Test Results @ http://nginx-thrift.social-network.svc.cluster.local:8080 
  Thread Stats   Avg      Stdev     99%   +/- Stdev
    Latency   245.36ms   32.43ms 408.06ms   90.85%
    Req/